<a href="https://colab.research.google.com/github/zaki-moh/MMA_fight_predictor/blob/main/mma_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ufc-fighters-statistics.csv to ufc-fighters-statistics.csv


In [ ]:
import pandas as pd
stats = pd.read_csv("ufc-fighters-statistics.csv")
stats.head()
stats.isnull().sum()
print(stats.head(10))
#print(stats.columns)

#

                name               nickname  wins  losses  draws  height_cm  \
0    Robert Drysdale                    NaN     7       0      0     190.50   
1  Daniel McWilliams             The Animal    15      37      0     185.42   
2         Dan Molina                    NaN    13       9      0     177.80   
3          Paul Ruiz                    NaN     7       4      0     167.64   
4    Collin Huckbody                 All In     8       2      0     190.50   
5   Gerald Strebendt  The Finishing Machine     9       7      0     175.26   
6        Isaiah Hill                    NaN     5       7      1     177.80   
7   Kenneth Seegrist                    NaN     4       7      0     182.88   
8          Will Kerr                    NaN     9       3      0     177.80   
9         Neil Grove                Goliath    12       8      1     198.12   

   weight_in_kg  reach_in_cm    stance date_of_birth  \
0         92.99          NaN  Orthodox    1981-10-05   
1         83.91   

In [ ]:
stats.info()
stats.describe()
stats.shape
print(stats.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4111 entries, 0 to 4110
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   name                                          4111 non-null   object 
 1   nickname                                      2257 non-null   object 
 2   wins                                          4111 non-null   int64  
 3   losses                                        4111 non-null   int64  
 4   draws                                         4111 non-null   int64  
 5   height_cm                                     3813 non-null   float64
 6   weight_in_kg                                  4024 non-null   float64
 7   reach_in_cm                                   2184 non-null   float64
 8   stance                                        3288 non-null   object 
 9   date_of_birth                                 2976 non-null   o

In [ ]:
stats['height_cm'] = stats['height_cm'].fillna(stats['height_cm'].median())
stats['date_of_birth'] = pd.to_datetime(stats['date_of_birth'], errors='coerce')
today = pd.to_datetime('today')
stats['age'] = (today - stats['date_of_birth']).dt.days // 365
stats['age'] = stats['age'].fillna(stats['age'].median())
mean_strikes = stats['significant_strikes_landed_per_minute'].mean()
stats['normalized_strikes'] = stats['significant_strikes_landed_per_minute'] / mean_strikes
stats['performance'] = (0.3 * stats['normalized_strikes']
                        + 0.3 * stats['significant_strike_defence'] / 100
                        + 0.2 * stats['takedown_accuracy'] / 100
                        + 0.2 * stats['takedown_defense'] / 100)
stats['strike_efficiency'] = (
    stats['significant_strikes_landed_per_minute']
    - stats['significant_strikes_absorbed_per_minute']
)
stats['grapple_efficiency'] = (
    0.5 * stats['takedown_accuracy'] / 100 + 0.5 * stats['takedown_defense'] / 100
)
stats['win_indicator'] = (stats['wins'] > stats['losses']).astype(int)
stats['win_ratio'] = stats['wins'] / (stats['wins'] + stats["losses"] + stats['draws'])
stats['win_ratio'] = stats['win_ratio'].fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_cols = ['height_cm', 'age', 'performance', 'strike_efficiency', 'grapple_efficiency']

stats[numeric_cols] = scaler.fit_transform(stats[numeric_cols])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = stats[['strike_efficiency', 'grapple_efficiency', 'height_cm', 'age', 'performance']]
y = stats['win_indicator']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8394941634241245
              precision    recall  f1-score   support

           0       0.49      0.54      0.51       160
           1       0.91      0.90      0.90       868

    accuracy                           0.84      1028
   macro avg       0.70      0.72      0.71      1028
weighted avg       0.85      0.84      0.84      1028



In [ ]:
def predictWinner(fighter_A_Name, fighter_B_Name):
  fighter_A = stats.loc[stats['name'] == fighter_A_Name].squeeze()
  fighter_B = stats.loc[stats['name'] == fighter_B_Name].squeeze()

  if fighter_A.empty or fighter_B.empty:
    print("One or both fighters not found in the dataset.")
    return

  features = ['strike_efficiency', 'grapple_efficiency', 'height_cm', 'age', 'performance']

  feature_A = fighter_A[features].values.reshape(1, -1)
  feature_B = fighter_B[features].values.reshape(1, -1)

  A_prob = model.predict_proba(feature_A)[0][1]
  B_prob = model.predict_proba(feature_B)[0][1]

  print(f"{fighter_A_Name} vs {fighter_B_Name}")
  print(f"{fighter_A_Name} likelyhood to win {A_prob:.2%}")
  print(f"{fighter_B_Name} likelyhood to win {A_prob:.2%}")

  if A_prob > B_prob:
    print(f"Predicted winner {fighter_A_Name}")
  else:
    print(f"Predicted winner {fighter_B_Name}")




In [ ]:
#Test
predictWinner("Khabib Nurmagomedov", "Conor McGregor")

Khabib Nurmagomedov vs Conor McGregor
Khabib Nurmagomedov likelyhood to win 100.00%
Conor McGregor likelyhood to win 100.00%
Predicted winner Conor McGregor


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
